### 비트코인 틱데이터 전처리 버전2
- num_rows 적은 것들 제거 + 그에 해당하는 sequence_length 전부 삭제
- num_rows = 5
- inf에 해당하는 값 제거

In [2]:
import pandas as pd
import numpy as np

In [3]:
file_path = '../../data/'
df = pd.read_csv(file_path + 'BTC_sum_both_30s_202303271051_v1.csv')
df.shape

(151838, 84)

In [4]:
# num_rows 컬럼의 값이 적은 것은 트레이드가 거의 없었다는 의미인데 이는 수집 과정에서 오류가 있었을 것 같기에 데이터 클리닝(제거) 작업이 필요하다.

lst = [len(df[df['num_rows'] <= i]) for i in range(10)]

num_rows_df = pd.DataFrame({
    'num_rows' : range(10),
    'counts' : lst,
    'proportion' : np.round(np.array(lst) / len(df), 3) * 100
})

num_rows_df

,num_rows,counts,proportion
0,0,0,0.0
1,1,514,0.3
2,2,1411,0.9
3,3,2734,1.8
4,4,4465,2.9
5,5,6462,4.3
6,6,8817,5.8
7,7,11419,7.5
8,8,14307,9.4
9,9,17344,11.4


In [6]:
# 음수, 양수 무한대 값 출력 (음수 무한대 값은 없음을 확인)
#df[df.isin([np.inf]).any(axis=1)]

# 무한대에 해당하는 값 제거
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.dropna(inplace=True)

In [7]:
# 전체 데이터의 4.3퍼센트에 해당하는 5개 이하에 해당하는 데이터 인덱스 Check
num = 5

# num_rows 값에 따라 0 또는 1의 값을 가지는 열 생성
df['del_idx'] = df['num_rows'].apply(lambda x: 1 if x <= num else 0) # num 이하이면 1, 아니면 0


In [8]:
# 데이터 내보내기
#df.to_csv(f'{file_path}bitcoin_data_num_rows_gt_{num}.csv', index=False)

In [5]:
# 삭제된(즉, 수집 과정에서 오류가 있었던) 데이터가 특정 시점에 몰려있는지 확인 -> 몰려 있다면 코드 문제가 아닌 그 당시에 네트워크, 서버, 그 외 요인이 작용했을 확률이 높다.

# datetime으로 변환
new_df['window_start'] = pd.to_datetime(new_df['window_start'], format='%Y-%m-%d %H:%M:%S')

# 데이터 포인트 사이의 시간 차이 계산 (수정된 부분)
new_df['time_diff'] = new_df['window_start'].diff()

# 데이터프레임의 인덱스를 재설정
new_df.reset_index(drop=True, inplace=True)

# 가장 긴 공백 찾기
max_gap_index = new_df['time_diff'].idxmax()

# 안전한 인덱스 접근을 위한 조건 추가
if max_gap_index > 0 and max_gap_index < len(new_df):
    max_gap_value = new_df.loc[max_gap_index, 'time_diff']
    max_gap_start = new_df.loc[max_gap_index - 1, 'window_start']
    max_gap_end = new_df.loc[max_gap_index, 'window_start']
else:
    max_gap_value = None
    max_gap_start = None
    max_gap_end = None

# 결과 출력
print(f"가장 긴 공백의 시작: {max_gap_start}, 끝: {max_gap_end}, 길이: {max_gap_value}")


가장 긴 공백의 시작: 2022-12-21 23:48:00, 끝: 2022-12-22 06:20:00, 길이: 0 days 06:32:00


/var/folders/yd/1_xwcyjj6z58p2vptxk1dwvm0000gn/T/ipykernel_84973/4123251106.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['window_start'] = pd.to_datetime(new_df['window_start'], format='%Y-%m-%d %H:%M:%S')
/var/folders/yd/1_xwcyjj6z58p2vptxk1dwvm0000gn/T/ipykernel_84973/4123251106.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['time_diff'] = new_df['window_start'].diff()


In [6]:
# 6시간 32분을 초 단위로 환산
total_seconds = (6 * 60 * 60) + (32 * 60)

# 30초로 나누기
result = total_seconds / 30

result

784.0